<a href="https://colab.research.google.com/github/JeLaKo/apple-tree-disease/blob/main/Milestone4_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the data and importing the necessary libraries

In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [7]:
import os
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import math as tfmath
from sklearn.model_selection import train_test_split
import seaborn as sns

/content/gdrive/MyDrive/MLP/train


In [8]:
# import mlxtend package for confusion matrix
import mlxtend
                                                          
print(mlxtend.__version__) 

! pip install mlxtend --upgrade --no-deps

print(mlxtend.__version__) 

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix


0.19.0
0.19.0


In [2]:
'''
Only run the following cells if data needs to be newly downloaded !
'''
# !pip install -U -q kaggle==1.5.8

In [5]:
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/MLP/"
# ! kaggle competitions download -c plant-pathology-2021-fgvc8

In [ ]:
# %cd /content/gdrive/MyDrive/MLP/

In [ ]:
# ! mkdir train

In [6]:
# ! unzip -q plant-pathology-2021-fgvc8.zip -d train

Data processing


In [ ]:
%cd /content/gdrive/MyDrive/MLP/train/

In [9]:
# read training data
df = pd.read_csv('train.csv')

In [10]:
# change combined classes to only contain a single label
new_labels = df['labels'].to_list()

for i in range(len(new_labels)):
  if new_labels[i] == 'scab frog_eye_leaf_spot complex' or new_labels[i] == 'scab frog_eye_leaf_spot':
    new_labels[i] = 'scab'
  elif new_labels[i] == 'frog_eye_leaf_spot complex':
    new_labels[i] = 'frog_eye_leaf_spot'
  elif new_labels[i] == 'powdery_mildew complex':
    new_labels[i] = 'powdery_mildew'
  elif new_labels[i] == 'rust complex' or new_labels[i] == 'rust frog_eye_leaf_spot':
    new_labels[i] = 'rust'

# replace labels with adjusted labels in dataframe
df['adjusted labels'] = np.array(new_labels)
df = df.drop('labels', axis = 1)

In [12]:
def sample(df, sample_size):
  """
  This function gets an equal sample inclusive of all classes from the input dataframe
  """
  df_sampled = []
  classes = df['adjusted labels'].unique()

  for i in classes:
      g = df[df['adjusted labels'] == i].sample(sample_size)
      df_sampled.append(g)

  df_sampled = pd.concat(df_sampled)
  return df_sampled

# select sample from dataframe
df_sampled = sample(df, 1184)
print(df_sampled.head(2))

                      image adjusted labels
17602  faaa83b7f029445a.jpg         healthy
11495  d0783ec96abc4a4d.jpg         healthy
5011   a72ce33a86f60be0.jpg         healthy
14342  e4a55870fc9f1394.jpg         healthy
17386  f909db508fe0867c.jpg         healthy
...                     ...             ...
2272   924dd81aed8372cd.jpg  powdery_mildew
10027  c56a369b0bd0e0f5.jpg  powdery_mildew
3155   9896e68da7e59524.jpg  powdery_mildew
17846  fc542b37e0e79940.jpg  powdery_mildew
3770   9f946a2a2d38d5b4.jpg  powdery_mildew

[7104 rows x 2 columns]


In [13]:
def one_hot(df):
  """
  This function returns all classes and combination of classes found in the input-dataframe, and returns
  the one-hot encoded version
  """
  one_hot = pd.get_dummies(df['adjusted labels'])
  df = df.drop('adjusted labels', axis = 1)
  df = df.join(one_hot)
  return df

# convert labels of dataframe to one-hot encoded classes
df_onehot = one_hot(df_sampled)
print(df_onehot.head(2))

Image-data processing

In [ ]:
'''
Only run this cell if images need resizing !
'''
! mkdir resized_train_images

# gather names of all images in the image-directory
train_images = os.listdir('train_images/')

# resize all images and save it to a new directory
# for faster performance
for image in train_images:
  img = cv2.imread('train_images/' + image)
  resized_img = cv2.resize(img, (96, 96)) 
  cv2.imwrite('resized_train_images/' + image, resized_img)


In [ ]:
# convert dataframe to a hashable list; dictionary
df_dict = df_onehot.set_index('image').T.to_dict('list')

# gather names of all images in the resized directory
resized_images = os.listdir('resized_train_images/')

images = []
# find corresponding image from the resized directory to the selected sample found
# in dictionary and add that to a list
for image in df_dict.keys():
  if image in resized_images:
    img_resized = cv2.imread('resized_train_images/' + image) 
    images.append(img_resized)

In [ ]:
# convert image list to array
images = np.array(images)

# convert dictionary values to array
labels = np.array(list(df_dict.values()))

# split data
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size = 0.3, random_state=42)

Building and training the network

In [14]:
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, preprocessing

def train_and_evaluate(model, train_x, train_y, val_x, val_y, preprocess={}, epochs=20, augment={}):

    # optimizer = keras.optimizers.Adam(lr = 0.01)
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

    train_gen = preprocessing.image.ImageDataGenerator(**preprocess, **augment)
    train_gen.fit(train_x) 

    val_gen = preprocessing.image.ImageDataGenerator(**preprocess)
    val_gen.fit(train_x)

    history = model.fit(train_gen.flow(train_x, train_y), epochs=epochs, 
                        validation_data=val_gen.flow(val_x, val_y))

    fig, axs = plt.subplots(1,2,figsize=(20,5)) 

    for i, metric in enumerate(['loss', 'accuracy']):
        axs[i].plot(history.history[metric])
        axs[i].plot(history.history['val_'+metric])
        axs[i].legend(['training', 'validation'], loc='best')

        axs[i].set_title('Model '+metric)
        axs[i].set_ylabel(metric)
        axs[i].set_xlabel('epoch')

    plt.show()

    print(f"Validation Accuracy: {model.evaluate(val_gen.flow(val_x, val_y))[1]}")

In [ ]:
# ML MODEL ARCHITECTURE
# Define Sequential model
model = models.Sequential()

# create convolutional layer and max pooling layer
model.add(layers.Conv2D(32, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same', input_shape=(96, 96, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# create convolutional layer (larger) and max pooling layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())

# add Conv2D layer with 128 filters and max pooling layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())

# add Conv2D layer with 256 filters and max pooling layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(layers.Conv2D(256, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# add Conv2D layer with 32 filters and max pooling layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(layers.Conv2D(32, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# flatten layers
model.add(layers.Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(layers.Dense(256, activation=tf.keras.layers.LeakyReLU(alpha=0.5)))

# apply softmax activation for final layer classification
model.add(tf.keras.layers.Dropout(0.2))
model.add(layers.Dense(6, activation='softmax'))

# normalize input data: set preprocesing dictionary
preprocess = {'featurewise_center': True, 'featurewise_std_normalization' : True}

# augment data: set augmentation dictionary
augment = {'horizontal_flip': True, 
           'vertical_flip': True, 
           'rotation_range': 20, 
           'width_shift_range': 0.1, 
           'height_shift_range': 0.1, 
           'zoom_range': [0.3,1.0], 
           'brightness_range': [0.2,1.2],
           'channel_shift_range' : 0.7}

# run training and evaluation function
train_and_evaluate(model, x_train, y_train, x_val, y_val, preprocess, epochs = 80, augment = augment)


In [ ]:
model.summary()

Confusion Matrix

In [ ]:
# select classes
classes = df['adjusted labels'].unique()
display(classes)

# gather actual and predicted classes
y_true = tf.argmax(y_val, axis=1)
# y_pred = tf.keras.utils.to_categorical(y_pred, num_classes = 6)
y_pred = tf.argmax(model.predict(x_val), axis = 1)

# plot confusion matrix 1
conf_matrix = tfmath.confusion_matrix(y_true, y_pred, num_classes = 6)

ax = sns.heatmap(conf_matrix, xticklabels=classes, yticklabels=classes)
ax.set(xlabel='Predicted Class', ylabel='Actual Class')
plt.show()

print(conf_matrix)

In [ ]:
# plot confusion matrix 2
mtrx = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(conf_mat = mtrx, figsize=(8, 8), class_names=classes, colorbar=True, show_normed = True)